In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/noshowappointments/KaggleV2-May-2016.csv


In [2]:
# !pip install imbalanced-learn

In [3]:
# check version number
import imblearn
print(imblearn.__version__)

0.9.0


In [4]:
from matplotlib import pyplot as plt
# from sklearn.preprocessing import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler,SMOTE, ADASYN
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix,auc,roc_auc_score,precision_score,recall_score
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

In [5]:
dataset = pd.read_csv('../input/noshowappointments/KaggleV2-May-2016.csv')

In [6]:
dataset.head()

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   PatientId       110527 non-null  float64
 1   AppointmentID   110527 non-null  int64  
 2   Gender          110527 non-null  object 
 3   ScheduledDay    110527 non-null  object 
 4   AppointmentDay  110527 non-null  object 
 5   Age             110527 non-null  int64  
 6   Neighbourhood   110527 non-null  object 
 7   Scholarship     110527 non-null  int64  
 8   Hipertension    110527 non-null  int64  
 9   Diabetes        110527 non-null  int64  
 10  Alcoholism      110527 non-null  int64  
 11  Handcap         110527 non-null  int64  
 12  SMS_received    110527 non-null  int64  
 13  No-show         110527 non-null  object 
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [8]:
dataset.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


In [9]:
dataset['No-show'].value_counts()

No     88208
Yes    22319
Name: No-show, dtype: int64

In [10]:
dataset.isnull().sum().any()

False

In [11]:
dataset.drop(['PatientId','AppointmentID','ScheduledDay', 'AppointmentDay'], axis=1, inplace=True)

In [12]:
X = dataset.iloc[:, 0:9].values
Y = dataset.iloc[:, 9].values

In [13]:
X

array([['F', 62, 'JARDIM DA PENHA', ..., 0, 0, 0],
       ['M', 56, 'JARDIM DA PENHA', ..., 0, 0, 0],
       ['F', 62, 'MATA DA PRAIA', ..., 0, 0, 0],
       ...,
       ['F', 21, 'MARIA ORTIZ', ..., 0, 0, 1],
       ['F', 38, 'MARIA ORTIZ', ..., 0, 0, 1],
       ['F', 54, 'MARIA ORTIZ', ..., 0, 0, 1]], dtype=object)

In [14]:
Y

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype=object)

In [15]:
from collections import Counter
# summarize class distribution
counter = Counter(Y)
print(counter)

Counter({'No': 88208, 'Yes': 22319})


In [16]:
Y.shape

(110527,)

In [17]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

le = LabelEncoder()
X[:, 0] = le.fit_transform(X[:, 0])
X[:, 2] = le.fit_transform(X[:, 2])
# X[:, 9] = le.fit_transform(X[:, 9])
# X[:, 13] = le.fit_transform(X[:, 13])

Y = le.fit_transform(Y)

In [18]:
# transform the dataset
oversample = SMOTE()
X, Y = oversample.fit_resample(X, Y)
# summarize the new class distribution
counter = Counter(Y)
print(counter)

Counter({0: 88208, 1: 88208})


In [19]:
print(X)
print(Y)

[[ 0.         62.         39.         ...  0.          0.
   0.        ]
 [ 1.         56.         39.         ...  0.          0.
   0.        ]
 [ 0.         62.         45.         ...  0.          0.
   0.        ]
 ...
 [ 1.         40.48449095 40.         ...  0.          0.
   0.        ]
 [ 1.          0.         74.         ...  0.          0.
   0.        ]
 [ 0.         77.55885466 20.11770932 ...  0.          0.
   0.        ]]
[0 0 0 ... 1 1 1]


In [20]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X = sc.fit_transform(X)

In [21]:
X

array([[0.        , 0.54310345, 0.4875    , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.49137931, 0.4875    , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.54310345, 0.5625    , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.35762492, 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.00862069, 0.925     , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.67723151, 0.25147137, ..., 0.        , 0.        ,
        0.        ]])

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

In [23]:
X_train.shape

(141132, 9)

In [24]:
X_test.shape

(35284, 9)

In [25]:
rf = RandomForestClassifier(n_estimators=100)

In [26]:
rf.fit(X_train,Y_train)

RandomForestClassifier()

In [27]:
predict = rf.predict(X_test)
predict

array([0, 1, 0, ..., 0, 0, 0])

In [28]:
print(classification_report(Y_test,predict))

              precision    recall  f1-score   support

           0       0.77      0.81      0.79     17505
           1       0.80      0.76      0.78     17779

    accuracy                           0.78     35284
   macro avg       0.78      0.78      0.78     35284
weighted avg       0.78      0.78      0.78     35284



In [29]:
confusion_matrix(Y_test,predict)

array([[14127,  3378],
       [ 4284, 13495]])